# 모듈 임포트

In [ ]:
pip install captum

In [3]:
torch.__version__

'1.9.1+cu111'

In [6]:
captum.__version__

'0.6.0'

In [2]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import random
import torch.optim as optim
from torch.optim import AdamW
from torch import nn
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.nn import functional as F
import re
import urllib.request
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from konlpy.tag import Mecab
import wandb
import collections
import pickle
import kss
from gensim.summarization.summarizer import summarize
from transformers import set_seed

from ktextaug import TextAugmentation
import ktextaug

import os
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForMaskedLM
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerActivation
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import captum
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'NanumGothic'

random_seed = 42
random.seed(random_seed)
set_seed(random_seed)
np.random.seed(random_seed)
os.environ["PYTHONHASHSEED"] = str(random_seed)
tf.random.set_seed(random_seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(random_seed)
if device == 'cuda':
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 1. 데이터 불러오기

In [4]:
def load_data(file_name):#파일이름만 넣기
    data_path = os.getenv('HOME') + '/aiffel/project/FnGuide/data/ad_news'#경로는 자신의 경로로 바꿔서 하면 됨
    file_path = os.path.join(data_path, file_name)
    df = pd.read_csv(file_path, encoding='utf-8', delimiter='\t')
    return df

## 사전학습 모델 및 토크나이저 불러오기

## kule/roberta-base

In [16]:
model_name = "klue/roberta-base"  # 허깅페이스에서 모델만 갈아끼우면 됨

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, output_attentions=True)  # 예측하고자 하는 클래스의 수를 넣어줘야함
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 학습시킨 모델 불러오기
model_save_folder = os.getenv('HOME') + "/aiffel/project/FnGuide/model"
model_path = os.getenv('HOME') + "/aiffel/project/FnGuide/model/klue_roberta-base_None_cross_entropy_stopword_classification_model_3.pt" 
model.load_state_dict(torch.load(model_path))

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classif

<All keys matched successfully>

## kfdeberta-base

In [8]:
model_name='kfdeberta-base'

p_model_path = os.getenv('HOME') + "/aiffel/project/FnGuide/kfdeberta-base"

model = AutoModelForSequenceClassification.from_pretrained(p_model_path, num_labels=2, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(p_model_path)

# 학습시킨 모델 불러오기
model_save_folder = os.getenv('HOME') + "/aiffel/project/FnGuide/model"
model_path = os.path.join(model_save_folder, "kfdeberta-base_None_cross_entropy_stopword_classification_model_3.pt")

model.load_state_dict(torch.load(model_path))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /aiffel/aiffel/project/FnGuide/kfdeberta-base and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [17]:
# 입력 문장 준비
input_text = """사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다."""
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

# 문장을 토큰화하여 인덱스로 변환하고, 토큰들을 리스트로 저장합니다.
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

In [18]:
# 모델을 평가 상태로 변경
model.eval()
with torch.no_grad():
    # base_model의 forward 함수를 호출하여 결과를 얻습니다.
    # output_attentions=True로 설정하여 어텐션 가중치를 가져옵니다.
    outputs = model(input_ids=input_ids, output_attentions=True)
    
    # 어텐션 가중치를 변수 attentions에 저장합니다.
    attentions = outputs.attentions

In [14]:
def visualize_token2token_scores(attentions: list, layer_idx: int):
    """
    어텐션 가중치 시각화 함수
    Input:
        attentions (List[Tensor]): 레이어별 어텐션 가중치가 담긴 리스트
        layer_idx (int): 시각화하고자 하는 레이어 인덱스

    Returns:
        None: 어텐션 가중치를 시각화하여 표시합니다.
    """
    
    # layer_idx번째 레이어에 해당하는 어텐션 가중치의 shape를 확인하여 머리의 개수를 얻습니다.
    num_heads = attentions[layer_idx].shape[1]

    fig, axes = plt.subplots(num_heads, 1, figsize=(30, 90))

    for head_idx in range(num_heads):
        # layer_idx번째 레이어의 head_idx번째 머리의 어텐션 가중치를 가져옵니다.
        scores_np = attentions[layer_idx][0, head_idx].detach().cpu().numpy()
        
        # subplot을 선택합니다. (만약 num_heads가 1이면 하나의 subplot만 사용합니다)
        ax = axes[head_idx] if num_heads > 1 else axes
        
        # 어텐션 가중치를 이미지 형태로 그립니다.
        im = ax.matshow(scores_np, cmap='viridis')

        fontdict = {'fontsize': 10}

        ax.set_xticks(range(len(all_tokens)))
        ax.set_yticks(range(len(all_tokens)))

        ax.set_xticklabels(all_tokens, fontdict=fontdict, rotation=90)
        ax.set_yticklabels(all_tokens, fontdict=fontdict)
        ax.set_xlabel('Layer {}, Head {}'.format(layer_idx + 1, head_idx + 1))

        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

### 원하는 레이어의 어텐션 맵 출력

In [ ]:
# klue/roberta-base
layer = 0
visualize_token2token_scores(attentions, layer)

In [ ]:
# kfdeberta-base
layer = 0
visualize_token2token_scores(attentions, layer)

---